Amazon and Yelp review Classification

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

In [3]:
data_yelp = pd.read_csv('/home/al/Projects_My/NLP-russian-language/datasets/amazon/yelp_labelled.txt', sep = '\t', header = None)

In [4]:
data_yelp.head()
#review and sentiment
# 0 -Negative, 1 - Positive for positive review 

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
# Assign column names
columan_name = ['Review', 'Sentiment']
data_yelp.columns = columan_name

In [6]:
data_yelp.shape

(1000, 2)

In [7]:
data_amazon = pd.read_csv('/home/al/Projects_My/NLP-russian-language/datasets/amazon/amazon_cells_labelled.txt', sep = '\t', header = None)
data_amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [8]:
data_amazon.columns = columan_name
data_amazon.shape

(1000, 2)

In [9]:
data_imbd = pd.read_csv('/home/al/Projects_My/NLP-russian-language/datasets/amazon/imdb_labelled.txt', sep = '\t', header = None)
data_imbd.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [10]:
data_imbd.columns = columan_name
data_imbd.shape

(748, 2)

In [12]:
data = data_yelp.append([data_amazon, data_imbd], ignore_index = True)
data.shape

(2748, 2)

In [13]:
# check distribution of sentiments
data['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [14]:
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [15]:
x = data['Review']
y = data['Sentiment']

Data Clearning

In [24]:
# here we will remove stopwords, punctuations
# as well as we will apply lemmatization
import string
import spacy
nlp = spacy.load('en_core_web_sm')
punct = string.punctuation
print(punct)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [25]:
from spacy.lang.en.stop_words import STOP_WORDS

In [26]:
stopwords = list(STOP_WORDS)

In [27]:
def text_data_clearning(sentence):
    doc = nlp(sentence)
    tokens = []
    for token in doc:
        if token.lemma_ != '-PRON-':
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens
        

In [28]:
text_data_clearning("Hello all, It's beatiful day outside there!")

['hello', 'beatiful', 'day', 'outside']

In [29]:
# Vectorization Feature Engineering (TF-IDF)
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [34]:
tfidf = TfidfVectorizer(tokenizer = text_data_clearning)
classifier = LinearSVC()

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)

In [36]:
x_train.shape, x_test.shape

((2198,), (550,))

In [37]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])
clf.fit(x_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_clearning at 0x7f1e98d3f160>)),
                ('clf', LinearSVC())])

In [38]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [42]:
y_pred = clf.predict(x_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[203  76]
 [ 49 222]]
              precision    recall  f1-score   support

           0       0.81      0.73      0.76       279
           1       0.74      0.82      0.78       271

    accuracy                           0.77       550
   macro avg       0.78      0.77      0.77       550
weighted avg       0.78      0.77      0.77       550



In [43]:
accuracy_score(y_test, y_pred)

0.7727272727272727